<a href="https://colab.research.google.com/github/MatteoFercia/chatbot_bert_gpt/blob/fine_tuning/QA_trasformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [ ]:
#opening the Json file
import json

In [ ]:
with open(r"train.json", "r") as read_file:
    train = json.load(read_file)

with open(r"test.json", "r") as read_file:
    test = json.load(read_file)

with open(r"predict.json", "r") as read_file:
    predict = json.load(read_file)

In [ ]:
train

[{'context': 'The room will be previously cleaned and sanitized according to the Stanza Semplice standards.Upon your arrival the room will be clean and sanitized through the use of disinfectants and ozone treatments that make the room safe and healthy.The cleaning company will have already carried out its task according to the most recent regulations related to the emergency from COVID-19.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'How my room will be cleaned before my arrival?',
    'answers': [{'text': 'The cleaning company will have already carried out its task according to the most recent regulations related to the emergency from COVID-19',
      'answer_start': 71}]}]},
 {'context': 'Our mission? Semplify the life of students.We from Stanza Semplice provide rooms for rent for students and we are at your side to make your university time straightforward and happy.Choosing the rightaccommodation is fundamental: it is the place where you will prepare for

In [ ]:
test

[{'context': 'Stanza Semplice in Cagliari has 83 rooms, prices starts from 295 euros to 440 euros',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'How many rooms can I find in the city of Cagliari?',
    'answers': [{'text': '83 rooms', 'answer_start': 38},
     {'text': "You'll find 83 rooms", 'answer_start': 74}]}]},
 {'context': "Contracts with Stanza Semplice are individual.Each tenant has his own contract with his rent and room; the other rooms do not affect your contractual position in any way.It's enough to have an identity document and an italian tax code.",
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'What documents do I need to sign the contracts?',
    'answers': [{'text': 'an identity document and an italian tax code',
      'answer_start': 21}]},
   {'id': '00003',
    'is_impossible': True,
    'question': 'The contract is for the entire apartment?',
    'answers': [{'text': 'No, Each tenant has his own contract with his r

In [ ]:
predict

[{'context': 'Bologna is the University City par excellence. If you are a student looking for a room to rent, Stanza Semplice can help you find it. Bologna is home to the oldest university in the western world, the Alma Mater Studiorum, which to this day remains one of the most prominent in Italy and the world. Stanza Semplice® in the city of Blogna has 105 rooms so far, prices starting from 360 euros to 900 euros',
  'qas': [{'question': 'How much is it the cost for a room in Bologna?',
    'id': '0'}]}]

In [ ]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
model_type="bert"
model_name= "bert-base-cased"

In [ ]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5


In [ ]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
model = QuestionAnsweringModel(model_type,model_name, args=train_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
model.train_model(train, eval_data=test)

convert squad examples to features:   0%|          | 0/4 [00:00<?, ?it/s]Could not find answer: 'Stanza Semplice standards.Upon your arrival the room will be clean and sanitized through the use of disinfectants and ozone treatments that make' vs. 'The cleaning company will have already carried out its task according to the most recent regulations related to the emergency from COVID-19'
Could not find answer: 'life of students.We from Stanza' vs. 'Semplify the life of students'
Could not find answer: 'Semplice provide rooms for rent for students and we are at your side to make your university time' vs. 'we have selected for you the best rooms in the most significant Italian universities cities'
add example index and unique id: 100%|██████████| 4/4 [00:00<00:00, 36954.22it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

correct,▁▁▁▁▁▁▁▁
eval_loss,██▅▃▂▂▁▁
global_step,▁▂▃▄▅▆▇█
incorrect,██████▁▁
similar,▁▁▁▁▁▁██
train_loss,███▆▅▃▁▁
correct,0
eval_loss,-0.32788
global_step,8
incorrect,1
similar,2


Running Epoch 0 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(5,
 {'global_step': [1, 2, 3, 4, 5],
  'correct': [0, 0, 0, 0, 0],
  'similar': [1, 1, 1, 1, 1],
  'incorrect': [2, 2, 2, 2, 2],
  'train_loss': [4.853515625,
   4.90234375,
   4.869140625,
   4.486328125,
   4.185546875],
  'eval_loss': [0.138671875,
   0.138671875,
   0.095947265625,
   0.07244873046875,
   0.062469482421875]})

In [ ]:
# Make predictions with the model
to_predict = [
    {
        "context": "Stanza Semplice® in the city of Bologna has 105 rooms, prices starting from 360 euros to 900 euros",
        "qas": [
            {
                "question": "How many rooms has Stanza Semplice in Bologna?",
                "id": "0",
            }
        ],
    }
]

In [ ]:
answers, probabilities = model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6797.90it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['empty']}]
